In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import model_selection
import re

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
data_df= pd.read_csv('/kaggle/input/cleaned-data-for-the-chatbot-collected-from-movies/dialogs_expanded.csv', index_col=False)

In [4]:
data_df

,Unnamed: 0,question,answer,question_as_int,answer_as_int,question_len,answer_len
0,1,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....,"[54, 67, 74, 74, 12, 1, 40, 1, 82, 70, 77, 83,...","[45, 77, 82, 1, 82, 70, 67, 1, 70, 63, 65, 73,...",71,55
1,2,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...,"[45, 77, 82, 1, 82, 70, 67, 1, 70, 63, 65, 73,...","[46, 73, 63, 87, 14, 14, 14, 1, 82, 70, 67, 76...",55,73
2,3,You're asking me out. That's so cute. What's ...,Forget it.,"[56, 77, 83, 8, 80, 67, 1, 63, 81, 73, 71, 76,...","[37, 77, 80, 69, 67, 82, 1, 71, 82, 14]",62,10
3,4,"No, no, it's my fault -- we didn't have a prop...",Cameron.,"[45, 77, 12, 1, 76, 77, 12, 1, 71, 82, 8, 81, ...","[34, 63, 75, 67, 80, 77, 76, 14]",65,8
4,9,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.,"[38, 77, 81, 70, 12, 1, 71, 68, 1, 77, 76, 74,...","[43, 67, 82, 1, 75, 67, 1, 81, 67, 67, 1, 85, ...",46,25
...,...,...,...,...,...,...,...
139404,221608,Well that one. The one who keeps looking at me.,ft could be you flatter yourself CoghilL It's ...,"[54, 67, 74, 74, 1, 82, 70, 63, 82, 1, 77, 76,...","[68, 82, 1, 65, 77, 83, 74, 66, 1, 64, 67, 1, ...",47,59
139405,221609,Choose your targets men. That's right Watch th...,Keep steady. You're the best shots of the Twen...,"[34, 70, 77, 77, 81, 67, 1, 87, 77, 83, 80, 1,...","[42, 67, 67, 78, 1, 81, 82, 67, 63, 66, 87, 14...",61,85
139406,221610,Colonel Durnford... William Vereker. I hear yo...,"Good ones, yes, Mr Vereker. Gentlemen who can ...","[34, 77, 74, 77, 76, 67, 74, 1, 35, 83, 80, 76...","[38, 77, 77, 66, 1, 77, 76, 67, 81, 12, 1, 87,...",74,60
139407,221611,"Your orders, Mr Vereker?",I'm to take the Sikali with the main column to...,"[56, 77, 83, 80, 1, 77, 80, 66, 67, 80, 81, 12...","[40, 8, 75, 1, 82, 77, 1, 82, 63, 73, 67, 1, 8...",24,56


In [5]:
data_df.drop(['Unnamed: 0','question_as_int','answer_as_int','question_len','answer_len'],axis=1,inplace=True)

In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139409 entries, 0 to 139408
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   question  139409 non-null  object
 1   answer    139409 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


In [7]:
data_df

,question,answer
0,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....
1,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...
2,You're asking me out. That's so cute. What's ...,Forget it.
3,"No, no, it's my fault -- we didn't have a prop...",Cameron.
4,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.
...,...,...
139404,Well that one. The one who keeps looking at me.,ft could be you flatter yourself CoghilL It's ...
139405,Choose your targets men. That's right Watch th...,Keep steady. You're the best shots of the Twen...
139406,Colonel Durnford... William Vereker. I hear yo...,"Good ones, yes, Mr Vereker. Gentlemen who can ..."
139407,"Your orders, Mr Vereker?",I'm to take the Sikali with the main column to...


In [8]:
data_df.head(10)

,question,answer
0,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....
1,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...
2,You're asking me out. That's so cute. What's ...,Forget it.
3,"No, no, it's my fault -- we didn't have a prop...",Cameron.
4,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.
5,C'esc ma tete. This is my head,Right. See? You're ready for the quiz.
6,That's because it's such a nice one.,Forget French.
7,How is our little Find the Wench A Date plan p...,"Well, there's someone I think might be --"
8,You have my word. As a gentleman,You're sweet.
9,Sure have.,"I really, really, really wanna go, but I can't..."


In [9]:
data_df.sample(10)

,question,answer
99475,Hi. Is Frank there?,I think you have the wrong number.
37183,That Foley looks like he's been through a war ...,I've seen better.
136853,Why are you here?,I don't know.
29397,You're not contagious are you?,I don't think so.
70432,...to die for a country that's...,...never done a thing for you?
40594,How long ago was your makeover?,My what?!
84563,Don't tell me I can't --,They're trying to kill me!
13092,I don't know. I was just curious.,Maybe yes. Maybe no.
138232,I'm fine. I'm going to live a long time. That'...,Johnny 
24617,That would be Beaumont Livingston.,That's him. How do you know 'em?


#### re is a package to take care of regular expressions in the data
##### re.sub is used to substitute the expression with given input

In [10]:
def clean_text(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('\n', '', text)
  text = re.sub(r'[^\w]',' ',text)
  text = re.sub('\w*\d\w*', '', text)
  return text

data_df.question = data_df.question.map(clean_text)
data_df.answer = data_df.answer.map(clean_text)

In [11]:
def add_start_end(text):
  text = f'<start> {text} <end>'
  return text

data_df.question = data_df.question.map(add_start_end)
data_df.answer = data_df.answer.map(add_start_end)

In [12]:
data_df

,question,answer
0,<start> well i thought we d start with pronun...,<start> not the hacking and gagging and spitti...
1,<start> not the hacking and gagging and spitti...,<start> okay then how bout we try out some...
2,<start> you re asking me out that s so cute ...,<start> forget it <end>
3,<start> no no it s my fault we didn t hav...,<start> cameron <end>
4,<start> gosh if only we could find kat a boyf...,<start> let me see what i can do <end>
...,...,...
139404,<start> well that one the one who keeps looki...,<start> ft could be you flatter yourself coghi...
139405,<start> choose your targets men that s right ...,<start> keep steady you re the best shots of ...
139406,<start> colonel durnford william vereker i...,<start> good ones yes mr vereker gentlemen ...
139407,<start> your orders mr vereker <end>,<start> i m to take the sikali with the main c...


#### Tokenizer is used for text pre processing
##### filters describe all the characters which are to be removed
##### oov_token is used for out of vocab words to be replaced by 'OOV'

##### After making seperate tokens we convert words into integer values using texts_to_sequence
##### pad_sequence is used to add padding to sequence to make them all same length

In [13]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', oov_token='<OOV>'
  )
  lang_tokenizer.fit_on_texts(lang)
  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
  return tensor, lang_tokenizer

In [14]:
question_sequence, question_tokenizer = tokenize(data_df.question)
answer_sequence, answer_tokenizer = tokenize(data_df.answer)

#### Splitting the data into training and test data

In [15]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(question_sequence, 
                answer_sequence, test_size = 0.1, random_state=42) 

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((125468, 29), (13941, 29), (125468, 32), (13941, 32))

##### For each element t in the tensor, it checks if t is not equal to 0 (assuming 0 is used as a padding token or a special token). If t is not 0, it prints the integer t and its corresponding word in the tokenizer's vocabulary using lang.index_word[t].


In [16]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print('%d---> %s' % (t, lang.index_word[t]))

print('Question')
convert(question_tokenizer, x_train[0])
print()
print('Answer')
convert(answer_tokenizer, y_train[0])

Question
2---> <start>
80---> yeah
13---> that
8---> s
11002---> blush
36---> on
32---> my
301---> wife
3384---> uses
9---> it
3---> <end>

Answer
2---> <start>
204---> ask
5535---> travis
22---> he
7---> s
6---> the
1765---> ladies
104---> man
3---> <end>


##### #vocab input size takes the size of input values
##### #tar size takes the size of target values
##### #embedding dimensionality specifies how many dimensions each word embedding vector will have
##### #units  specifies the number of units (neurons) in the recurrent layers of the sequence-to-sequence model
##### #batch size  refers to the number of input sequences processed together during training

In [17]:
vocab_inp_size = len(question_tokenizer.word_index)+1
vocab_tar_size =  len(answer_tokenizer.word_index)+1
embedding_dim = 256
units = 1024
batch_size=32

##### The following code creates train dataset and test dataset containing a pair from x and y
##### to randomize the data shuffle is used and the batches of 32 each are prepared to be processed together
##### prefetch allows tensorflow to pick the number of batches to be processed at a time according to the resources

In [18]:
def create_dataset(x, y, batch_size=32):
  data = tf.data.Dataset.from_tensor_slices((x, y))

  data = data.shuffle(1028)
  data = data.batch(batch_size, drop_remainder=True)

  data = data.prefetch(tf.data.experimental.AUTOTUNE)

  return data

train_dataset = create_dataset(x_train, y_train)
test_dataset = create_dataset(x_test, y_test)

In [19]:
  for q, a in train_dataset.take(1):
    print(f'Question:{q.shape}\n{q}')
  
    print(f'Answer:{a.shape}\n{a}')

Question:(32, 29)
[[    2   629    60    10  1922    67   417    14    24    15    32   493
   3534     3     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2   364    30   776     3     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2    33   167     4     3     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2    16    55     5    56     3     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2   159    16    10   155     3     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2    18  1070     9     8     6  2026     3     0     0     0     0
   

##### The encoder is responsible for processing the input sequence and producing an encoding or hidden state that contains information about the input sequence.
##### embedding converts the input integers into dense vectors and masks the padding used 
##### A GRU (Gated Recurrent Unit) layer, which is a type of recurrent neural network (RNN) layer. It is configured to return both sequences and the final state of the encoder. The recurrent_initializer='glorot_uniform' parameter sets the initializer for the recurrent weights.


In [20]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size):
      super(Encoder, self).__init__()

      self.batch_size = batch_size
      self.encoder_units = encoder_units
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
      self.gru = tf.keras.layers.GRU(self.encoder_units, 
                                           return_sequences=True,
                                           return_state=True,
                                           recurrent_initializer = 'glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_size, self.encoder_units))

##### The decoder is responsible for generating output sequences based on the encoded input information.
##### It takes the hidden layer as input or to remember the initial state
#### The decoder's output is compared to the target sequences to compute loss, and during inference, it generates sequences one step at a time based on previously generated tokens and the decoder's internal state.

In [21]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, decoder_units, batch_size):
      super(Decoder, self).__init__()

      self.batch_size = batch_size
      self.decoder_units = decoder_units
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
      self.gru = tf.keras.layers.GRU(self.decoder_units, 
                                           return_sequences=True,
                                           return_state=True,
                                           recurrent_initializer = 'glorot_uniform')
      
      self.fc = tf.keras.layers.Dense(vocab_size)


  def call(self, x, hidden):
    x = self.embedding(x)
    output, hidden = self.gru(x, initial_state = hidden)
    output = tf.reshape(output, (-1, output.shape[2]))
    x =  tf.nn.softmax(self.fc(output))
    return x, hidden

In [22]:
# vocab_inp_size = len(eng_tokenizer.word_index)+1
# vocab_tar_size =  len(spn_tokenizer.word_index)+1
# embedding_dim = 256
# units = 1024
# batch_size=32

encoder = Encoder(vocab_inp_size, embedding_dim, units, batch_size)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(q, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (32, 29, 1024)
Encoder Hidden state shape: (batch size, units) (32, 1024)


In [23]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, batch_size)

sample_decoder_output, _ = decoder(tf.random.uniform((batch_size, 1)), sample_hidden)

print ('Decoder output shape: (batch size, vocab_size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch size, vocab_size) (32, 27849)


##### (tf.keras.optimizers.Adam()) is a popular optimization algorithm for training deep neural networks. The optimizer is used to update the model's weights during training, aiming to minimize the defined loss function.
##### The loss function chosen here is SparseCategoricalCrossentropy, which is often used for tasks where each target sequence is a sequence of integers 

##### The loss_object is applied to compute the loss, and it results in a loss tensor of the same shape as the input.The loss tensor is element-wise multiplied by the mask, effectively zeroing out the loss contributions from padding tokens. Finally, it computes the mean loss across the batch dimension, which provides the average loss per batch while ignoring padding tokens.

In [24]:
# create the optimizer using the Adam optimizer
optimizer = tf.keras.optimizers.Adam()
# create the loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction='none')

# define the loss function for the training
def loss_function(real, pred):
  # create the mask to ignore the padding tokens
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  # mask shape == (batch_size, sequence_length)
  # calculate the loss
  loss_ = loss_object(real, pred)
  # mask the loss
  # how the mask works:
  # if the value is 1, the loss is calculated
  # if the value is 0, the loss is ignored
    #[1,1,1,1,1,1,0,0,0,0,0] mask
    # *
    #[2,6,2,1,6,3,2,1,5,7,9] input
    # =
    #[2,6,2,1,6,3,0,0,0,0,0] output
  mask = tf.cast(mask, dtype=loss_.dtype)
  # mask shape == (batch_size, sequence_length)

  loss_ *= mask
  # calculate the average loss per batch 
  return tf.reduce_mean(loss_)

In [25]:
# create the training metric 
train_loss = tf.metrics.Mean(name='train loss')
# create the testing metric 
test_loss =tf.metrics.Mean(name='test loss')

####  training step is designed to be used within a training loop to update the model's weights based on a batch of training data.


In [26]:
# create the training step
# using the tf.function decorator to speed up the training process by converting the training function to a TensorFlow graph
@tf.function
# define the training step 
def train_step(inputs, target, enc_hidden):
  # the encoder_hidden is the initial hidden state of the encoder
  # enc_hidden shape == (batch_size, hidden_size)

  # inilaize the loss to zero
  loss = 0
  # create the gradient tape to record the gradient of the loss with respect to the weights

  with tf.GradientTape() as tape:
    # pass the input to the encoder
    # enc_output shape == (batch_size, 49, hidden_size)
    # enc_hidden shape == (batch_size, hidden_size)
    # using the encoder to get the encoder_output and the encoder_hidden
    # using the encoder_hidden as the initial hidden state of the decoder
    enc_output, enc_hidden = encoder(inputs, enc_hidden)
    # set the initial decoder hidden state to the encoder hidden state
    dec_hidden = enc_hidden

    # create the start token 
    # start_token shape == (batch_size, 1)
    # repeat the start token for the batch size times
    dec_input = tf.expand_dims([answer_tokenizer.word_index['<start>']] * inputs.shape[0], 1)
    
    # Teacher forcing - feeding the target as the next input
    
    for t in range(1, target.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)
      # calculate the loss for the current time step using the loss function
      loss += loss_function(target[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(target[:, t], 1)
  # calculate the loss for the current batch
  batch_loss = (loss / int(target.shape[1]))

  # get the trainable variables
  variables = encoder.trainable_variables + decoder.trainable_variables
  # calculate the gradients using the tape 
  gradients = tape.gradient(loss, variables)
  # update the trainable variables
  optimizer.apply_gradients(zip(gradients, variables))
  # add the loss to the training loss metric
  train_loss(batch_loss)
  return batch_loss

In [27]:
# create the training step
# using the tf.function decorator to speed up the training process by converting the training function to a TensorFlow graph
@tf.function 
def test_step(inputs, target, enc_hidden):
    # the encoder_hidden is the initial hidden state of the encoder
    # enc_hidden shape == (batch_size, hidden_size)
    # inilaize the loss to zero
    loss = 0
    # pass the input to the encoder 
    # enc_output shape == (batch_size, 49, hidden_size) 
    # enc_hidden shape == (batch_size, hidden_size)
    # using the encoder to get the encoder_output and the encoder_hidden
    enc_output, enc_hidden = encoder(inputs, enc_hidden)
    # set the initial decoder hidden state to the encoder hidden state
    dec_hidden = enc_hidden
    # create the start token
    # start_token shape == (batch_size, 1)
    # repeat the start token for the batch size times
    dec_input = tf.expand_dims([answer_tokenizer.word_index['<start>']] * inputs.shape[0], 1)
    for t in range(1, target.shape[1]):
        # passing enc_output to the decoder with dec_hidden as the initial hidden state
        predictions, dec_hidden = decoder(dec_input, dec_hidden)
        # calculate the loss for the current time step using the loss function 
        loss += loss_function(target[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(target[:, t], 1)
    # calculate the loss for the current batch
    batch_loss = (loss / int(target.shape[1]))
    # add the batch loss to the test loss metric
    test_loss(batch_loss)

In [28]:
# set the epochs to 10
EPOCHS = 10
# set the old test loss to high number 

old_test_loss=1000000
# create the training loop
for epoch in range(EPOCHS):
    # reset the training loss metric
    train_loss.reset_states()
    # reset the testing loss metric
    test_loss.reset_states()

    # initalize the hidden state of the encoder to zeros 
    enc_hidden = encoder.initialize_hidden_state()
    # create the training progress bar set the total number of batches to the length of the training dataset and the batch size to the test size
    steps_per_epoch = answer_sequence.shape[0]//batch_size #=> 4356 batch in the dataset 
    bar = tf.keras.utils.Progbar(target=steps_per_epoch)
    
    count=0
    # iterate over the training dataset 
    for (batch, (inputs, target)) in enumerate(train_dataset):
        # update the progress bar
     count += 1
        # run the training step
     batch_loss = train_step(inputs, target, enc_hidden)
     bar.update(count)  # manually update the progress bar
                                                  
    
    
    # iterate over the testing dataset    
    for (batch, (inputs, target)) in enumerate(test_dataset):
     count += 1
        # run the testing step
     batch_loss = test_step(inputs, target, enc_hidden)
    bar.update(count)
    # save the best performance model on the test dataset 
    
    if old_test_loss> test_loss.result():
        # set the old test loss to the test loss 
        old_test_loss= test_loss.result()
        encoder.save(filepath='/content/models/encoder')
        decoder.save(filepath='/content/models/decoder')
        print('Model is saved')
    # print the training and testing loss
    print('#' * 50)
    print(f'Epoch #{epoch + 1}')
    print(f'Training Loss {train_loss.result()}')
    print(f'Testing Loss {test_loss.result()}')
    print('#' * 50)

4355/4356 [============================>.] - ETA: 0s  Model is saved
##################################################
Epoch #1
Training Loss 1.4245524406433105
Testing Loss 1.2976245880126953
##################################################
4355/4356 [============================>.] - ETA: 0s  Model is saved
##################################################
Epoch #2
Training Loss 1.2033432722091675
Testing Loss 1.2872092723846436
##################################################
4355/4356 [============================>.] - ETA: 0s  ##################################################
Epoch #3
Training Loss 1.0616378784179688
Testing Loss 1.3278298377990723
##################################################
4355/4356 [============================>.] - ETA: 0s  ##################################################
Epoch #4
Training Loss 0.9248971343040466
Testing Loss 1.3802748918533325
##################################################
4355/4356 [============================>.] - ETA: 

In [35]:
# create the chatbot function
# the chatbot function takes in the question as input and answers the input sentence 
def chatbot(sentence):
  
  # clean the input question sentence 
  sentence = clean_text(sentence)
  # add the start token to the sentence
  sentence =add_start_end(sentence)
  # tokenize the sentence
  inputs = question_tokenizer.texts_to_sequences([sentence])
  # pad the sentence
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                         maxlen=29,
                                                         padding='post')
  
  # initalize the hidden state of the encoder to zeros
  hidden = [tf.zeros((1, units))]
  # pass the sentence to the encoder with the hidden state as the initial hidden state
  enc_out, enc_hidden = encoder(inputs, hidden)
  # set the initial decoder hidden state to the encoder hidden state
  dec_hidden = enc_hidden
  # create the start token
  # start_token shape == (batch_size, 1)
  # repeat the start token for the batch size times
  dec_input = tf.expand_dims([answer_tokenizer.word_index['<start>']], 0)
  # create the result string
  result = ''
  # loop over the length of the sentence (32)

  for t in range(32):
    # passing the encoder output and the decoder hidden state to the decoder make sure the decoder input is the previous predicted word
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # getting the predicted word index
    predicted_id = tf.argmax(predictions[0]).numpy()
    # getting the predicted word using the predicted index
    # add the predicted word to the result string 
    result += answer_tokenizer.index_word[predicted_id] + ' '
    # if the predicted word is the <end> token then stop the loop
    if answer_tokenizer.index_word[predicted_id] == '<end>':
      # remove the <start> and <end> tokens from the result string
      result = result.replace('<start> ', '')
      result = result.replace(' <end> ','')
      # remove the <start> and <end> tokens from the sentence string
      sentence = sentence.replace('<start> ', '')
      sentence = sentence.replace(' <end>', '')
      return  sentence, result

    # using the predicted word as the next decoder input
    dec_input = tf.expand_dims([predicted_id], 0)
  # remove the <start> and <end> tokens from the result string
  result = result.replace('<start> ', '')
  result = result.replace('<end>','')
  # remove the <start> and <end> tokens from the sentence string
  sentence = sentence.replace('<start> ', '')
  sentence = sentence.replace('<end>', '')
  

  
 
  
  # return the result string and the original sentence
  return sentence, result

In [30]:
chatbot("how are you today")

('how are you today', 'i ll be there')

In [31]:
chatbot('what is the weather outside')


('what is the weather outside', 'the jewish disease')

In [32]:
chatbot('can you run')

('can you run', 'sure i can but that s down')

In [33]:
chatbot(' how old ')

(' how old ', 'i don t know')

In [34]:
chatbot('can you play')

('can you play', 'oh yes jeffrey')